In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn.utils
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from numpy import array
from numpy import asarray
from numpy import zeros

df1_type=pd.read_csv("event_type.csv",encoding='latin1')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramyajoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df2_type=df1_type.iloc[ : ,[1]]
df2_type=df2_type.replace(np.nan,'')
l1=list()
l1=df2_type.values.tolist()
df2_type['total']=pd.Series(l1).values
df2_type['total']=df2_type['total'].apply(set)

df2_type


,Type,total
0,Certifications,{Certifications}
1,Certifications,{Certifications}
2,Certifications,{Certifications}
3,Certifications,{Certifications}
4,Certifications,{Certifications}
...,...,...
1078,Webinars,{Webinars}
1079,Courses,{Courses}
1080,Expos,{Expos}
1081,Webinars,{Webinars}


In [8]:
df1_domain=pd.read_csv("event_domainfinal.csv",encoding='latin1')
df2_domain=df1_domain.iloc[ : ,[1]]
df2_doamin=df2_domain.replace(np.nan,'')
l2=list()
l2=df2_domain.values.tolist()
df2_domain['total']=pd.Series(l2).values
#print(df2)
df2_domain['total']=df2_domain['total'].apply(set)
df2_domain

,Domain,total
0,Management,{Management}
1,Management,{Management}
2,Management,{Management}
3,Management,{Management}
4,Management,{Management}
...,...,...
1077,Other,{Other}
1078,Other,{Other}
1079,Other,{Other}
1080,Other,{Other}


In [9]:
mlb = MultiLabelBinarizer()
l_type=mlb.fit_transform(df2_type['total'])
df3_type=pd.DataFrame(l_type)
df3_type.columns=list(mlb.classes_)
#df3=df3.drop(columns=[''])
print(mlb.classes_)
data=pd.Series(df1_type['Event'])
df3_type.insert(0,'Event',data)
df3_type = sklearn.utils.shuffle(df3_type)
df3_type = df3_type.reset_index(drop=True)
print(df3_type.head())


l_domain=mlb.fit_transform(df2_domain['total'])
df3_domain=pd.DataFrame(l_domain)
df3_domain.columns=list(mlb.classes_)
#df3=df3.drop(columns=[''])
print(mlb.classes_)
data=pd.Series(df1_domain['Event'])
df3_domain.insert(0,'Event',data)
df3_domain = sklearn.utils.shuffle(df3_domain)
df3_domain = df3_domain.reset_index(drop=True)
print(df3_domain.head())


['Certifications' 'Competitions' 'Courses' 'Expos' 'Fests' 'Hackathon'
 'Internships' 'Jobs' 'Talks' 'Training' 'Webinars' 'Workshops']
                                               Event  Certifications  \
0                  Spring cloud training- Free demo                0   
1  Mindmajix Provides .NET Training Online with a...               0   
2                              Beginner Code Champ 7               0   
3                  Smart city festival is awaiting!!               0   
4   Workshop - Kerala State Higher Education Council               0   

   Competitions  Courses  Expos  Fests  Hackathon  Internships  Jobs  Talks  \
0             0        0      0      0          0            0     0      0   
1             0        0      0      0          0            0     0      0   
2             0        0      0      0          1            0     0      0   
3             0        0      0      1          0            0     0      0   
4             0        0      0     

In [5]:

def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence =str(re.sub('[^0-9a-zA-Z]', ' ',str(sen)))

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
   
    sentence=str.lower(sentence)
    
    text_tokens = word_tokenize(sentence)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    filtered_sentence = (" ").join(tokens_without_sw)

    return filtered_sentence

In [10]:
x_type = []
x_domain=[]
sentences1 = list(df3_type["Event"])
sentences2 = list(df3_domain["Event"])
for sen in sentences1:
    x_type.append(preprocess_text(sen))
for sen in sentences2:
    x_domain.append(preprocess_text(sen))


    


In [11]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(x_type)
X_data_type = tokenizer1.texts_to_sequences(x_type)
vocab_size1 = len(tokenizer1.word_index) + 1

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(x_domain)
X_data_domain = tokenizer2.texts_to_sequences(x_domain)
vocab_size2 = len(tokenizer2.word_index) + 1

maxlen = 200
X_data_type = pad_sequences(X_data_type, padding='post', maxlen=maxlen)
X_data_domain = pad_sequences(X_data_domain, padding='post', maxlen=maxlen)
print(X_data_type.shape)
print(X_data_domain.shape)

(1083, 200)
(1082, 200)


In [12]:


event_type=df3_type[['Certifications','Competitions','Courses','Expos','Fests','Hackathon','Internships','Jobs','Talks','Training','Webinars','Workshops']]
event_domain=df3_domain[['Artificial Intelligence' ,'Blockchain', 'C' ,'C++' ,'Cloud Computing',
 'Coding', 'Data Science', 'Development Process', 'Finance' ,'Hardware',
 'Higher Education' ,'IoT', 'Java' ,'Javascript', 'Machine Learning',
 'Management' ,'Mobile Applications', 'Networking' ,'Other','Python',
 'Security' ,'Software Architecture', 'Web Development']]
y_type = event_type.values
y_domain = event_domain.values
print(y_type.shape)
print(y_domain.shape)


(1083, 12)
(1082, 23)


In [13]:
embeddings_dictionary = dict()

glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [14]:
embedding_matrix1 = zeros((vocab_size1, 100))
for word, index in tokenizer1.word_index.items():
    embedding_vector1 = embeddings_dictionary.get(word)
    if embedding_vector1 is not None:
        embedding_matrix1[index] = embedding_vector1

In [15]:
embedding_matrix2 = zeros((vocab_size2, 100))
for word, index in tokenizer2.word_index.items():
    embedding_vector2 = embeddings_dictionary.get(word)
    if embedding_vector2 is not None:
        embedding_matrix2[index] = embedding_vector2

In [16]:
input1 = tf.keras.layers.Input(shape=(maxlen,))
 
x1 = tf.keras.layers.Embedding(vocab_size1,100, weights=[embedding_matrix1], trainable=False)(input1)

x1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.1,
                                                      recurrent_dropout=0.1))(x1)
 
x1= tf.keras.layers.Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x1)
 
avg_pool1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
max_pool1 = tf.keras.layers.GlobalMaxPooling1D()(x1)
 
x1 = tf.keras.layers.concatenate([avg_pool1, max_pool1])
 
preds1 = tf.keras.layers.Dense(12, activation="sigmoid")(x1)
 
model1 = tf.keras.Model(input1, preds1)
 
#model1.summary()

In [96]:
model1.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
history1 = model1.fit(X_data_type, y_type,epochs=5, validation_split=0.2,class_weight='balanced')

Train on 866 samples, validate on 217 samples
Epoch 1/5
866/866 [==============================] - 51s 59ms/sample - loss: 0.0755 - accuracy: 0.9763 - val_loss: 0.0833 - val_accuracy: 0.9731
Epoch 2/5
866/866 [==============================] - 47s 54ms/sample - loss: 0.0591 - accuracy: 0.9809 - val_loss: 0.0768 - val_accuracy: 0.9758
Epoch 3/5
866/866 [==============================] - 48s 55ms/sample - loss: 0.0477 - accuracy: 0.9848 - val_loss: 0.0725 - val_accuracy: 0.9750
Epoch 4/5
866/866 [==============================] - 49s 57ms/sample - loss: 0.0391 - accuracy: 0.9882 - val_loss: 0.0692 - val_accuracy: 0.9785
Epoch 5/5
866/866 [==============================] - 50s 58ms/sample - loss: 0.0316 - accuracy: 0.9901 - val_loss: 0.0651 - val_accuracy: 0.9793


In [18]:
input2 = tf.keras.layers.Input(shape=(maxlen,))
 
x2 = tf.keras.layers.Embedding(vocab_size2,100, weights=[embedding_matrix2], trainable=False)(input2)

x2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True, dropout=0.1,
                                                      recurrent_dropout=0.1))(x2)
 
x2 = tf.keras.layers.Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x2)
 
avg_pool2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
max_pool2 = tf.keras.layers.GlobalMaxPooling1D()(x2)
 
x2 = tf.keras.layers.concatenate([avg_pool2, max_pool2])
 
preds2 = tf.keras.layers.Dense(23, activation="sigmoid")(x2)
 
model2 = tf.keras.Model(input2, preds2)
 
#model1.summary()

In [19]:
model2.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
history2 = model2.fit(X_data_domain, y_domain,epochs=5, validation_split=0.2,class_weight='balanced')

Train on 865 samples, validate on 217 samples
Epoch 1/5
865/865 [==============================] - 78s 91ms/sample - loss: 0.3589 - accuracy: 0.8679 - val_loss: 0.1993 - val_accuracy: 0.9565
Epoch 2/5
865/865 [==============================] - 61s 70ms/sample - loss: 0.1792 - accuracy: 0.9565 - val_loss: 0.1757 - val_accuracy: 0.9565
Epoch 3/5
865/865 [==============================] - 60s 69ms/sample - loss: 0.1612 - accuracy: 0.9565 - val_loss: 0.1609 - val_accuracy: 0.9565
Epoch 4/5
865/865 [==============================] - 61s 70ms/sample - loss: 0.1406 - accuracy: 0.9572 - val_loss: 0.1420 - val_accuracy: 0.9575
Epoch 5/5
865/865 [==============================] - 62s 72ms/sample - loss: 0.1182 - accuracy: 0.9605 - val_loss: 0.1265 - val_accuracy: 0.9613


In [140]:
tokenizer = Tokenizer()
labels_domain=['Artificial Intelligence' ,'Blockchain', 'C' ,'C++' ,'Cloud Computing',
 'Coding', 'Data Science', 'Development Process', 'Finance' ,'Hardware',
 'Higher Education' ,'IoT', 'Java' ,'Javascript', 'Machine Learning',
 'Management' ,'Mobile Applications', 'Networking','Other','Python',
 'Security' ,'Software Architecture', 'Web Development']

labels_type=['Certifications','Competitions','Courses','Expos','Fests','Hackathon','Internships','Jobs','Talks','Training','Webinars','Workshops']
def classify_event_type(x) :
    event_set=pd.read_csv(x)
    sentences = list(event_set["Event"])
    xvar=[]
    type_list=[]
    domain_list=[]
    for sen in sentences:
        xvar.append(preprocess_text(sen))
    #sen=preprocess_text(sen)
    #sen=np.array([sen])
    tokenizer.fit_on_texts(xvar)

    sen_data = tokenizer.texts_to_sequences(xvar)

    maxlen = 200

    sen_data = pad_sequences(sen_data, padding='post', maxlen=maxlen)
    #print(sen_data.shape)
    types=model1.predict(sen_data)
    domains=model2.predict(sen_data)
    
    list1=np.argmax(types,axis=1)
    list2=np.argmax(domains,axis=1)
    for var in list1:
        type_list.append(labels_type[var])
    for var in list2:
        domain_list.append(labels_domain[var])
    #match(type_list,domain_list)
    print(type_list)
    print(domain_list)
    emp_list=match(type_list,domain_list)
    c=[]
    for i in range(len(emp_list)):
        c.append(", ".join(emp_list[i]))
                   
        event_set['Name']=pd.Series(c)
    #print(event_set)
    event_set.to_excel("employee_list.xlsx")
    

In [141]:
def match(type_list,domain_list) :
    emp_list=[]
    emp=pd.read_csv("CCMLEmployeeData.csv")
    for x in range(3) :
        z=[];
        for index, row in emp.iterrows():
            if(domain_list[x]!='Other'):
                if((row['Event1']==type_list[x] or row['Event2']==type_list[x])and row['Domain']==domain_list[x]):
                    z.append(row['Name'])
            else :
                if(row['Event1']==type_list[x] or row['Event2']==type_list[x]):
                    z.append(row['Name'])


        emp_list.append(z)

    return(emp_list)


In [142]:
classify_event_type("test.csv")

[[6.33868040e-04 1.04583160e-04 6.33937214e-03 2.37145039e-04
  9.02451866e-05 4.59352555e-03 3.05002229e-03 4.02053818e-04
  2.52485261e-05 9.82651174e-01 8.46899784e-05 2.73429346e-03]
 [2.90888478e-03 5.28503326e-04 3.31001123e-04 9.52814706e-04
  7.35225194e-06 1.55606722e-05 7.71391788e-04 4.36904338e-05
  8.14802130e-04 4.29505249e-03 4.77907388e-03 9.89328444e-01]
 [8.22361946e-01 1.65787583e-07 2.14152824e-04 5.24841380e-06
  6.99097939e-07 6.04337920e-03 5.31786878e-04 2.02908132e-06
  1.12158254e-04 1.08808326e-03 1.17102936e-01 1.00177720e-04]]
['Training', 'Workshops', 'Certifications']
['Management', 'Networking', 'Javascript']
                                               Event  \
0                       C++ hackathon is live now!!!   
1                          Workshop on data science    
2  EmbeddedTech India expo 2020 is from 21 march ...   

                                                Name  
0                         John Pearson, Michael Rowe  
1               